In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install protobuf==3.20.0

In [ ]:
!pip install dlib
!pip install opencv-python

In [ ]:
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -d shape_predictor_68_face_landmarks.dat.bz2  # Unzip the file

In [ ]:
# Image preprocessing

In [ ]:
import os
import tensorflow as tf
import numpy as np
import cv2

def preprocess_image(image_path):
    # 1. Image Loading and Resizing
    image = cv2.imread(image_path)
    image = cv2.resize(image, (224, 224))

    # 2. Normalization (Scaling to [0, 1])
    image = image / 255.0

    # 5. Image Denoising using Gaussian Blur
    image = cv2.GaussianBlur(image, (5, 5), 0)

    # 6. Histogram Equalization
    gray_image = cv2.cvtColor((image * 255).astype(np.uint8), cv2.COLOR_RGB2GRAY)
    equalized_image = cv2.equalizeHist(gray_image)
    equalized_image = cv2.cvtColor(equalized_image, cv2.COLOR_GRAY2RGB) / 255.0

    # 8. Data Type Conversion
    image = equalized_image.astype(np.float32)

    return image

def process_images(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)

            try:
                processed_image = preprocess_image(input_path)
                # Convert to 8-bit and save the image
                cv2.imwrite(output_path, (processed_image * 255).astype(np.uint8))
                print(f"Processed and saved: {filename}")
            except Exception as e:
                print(f"Error processing {filename}: {e}")

# Paths to input and output folders
input_folder = '/content/drive/MyDrive/CAPSTONE_DATASETS/Autism_Images/data/train/Non_Autistic'
output_folder = '/content/drive/MyDrive/preprocessing prototype 2/train/Non_Autistic'

# Process all images
process_images(input_folder, output_folder)

In [ ]:
# Data augmentation

In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm

# Input: Grayscale images (output of previous script)
input_path = '/content/drive/MyDrive/CAPSTONE_DATASETS/Autism_Images/data/train/Autistic'
augmented_path = '/content/drive/MyDrive/preprocessing prototype 2/train/Augmented/Autistic'
os.makedirs(augmented_path, exist_ok=True)

def augment_and_save(image, filename, count):
    base_name = os.path.splitext(filename)[0]

    # Save original grayscale image
    cv2.imwrite(os.path.join(augmented_path, f"{base_name}_orig.jpg"), image)

    # Horizontal Flip
    flipped = cv2.flip(image, 1)
    cv2.imwrite(os.path.join(augmented_path, f"{base_name}_flip.jpg"), flipped)

    # Rotation
    for angle in [15, -15]:
        (h, w) = image.shape
        center = (w // 2, h // 2)
        M = cv2.getRotationMatrix2D(center, angle, 1.0)
        rotated = cv2.warpAffine(image, M, (w, h), borderMode=cv2.BORDER_REFLECT)
        cv2.imwrite(os.path.join(augmented_path, f"{base_name}_rot{angle}.jpg"), rotated)

    # Zoom
    for zoom_factor in [1.1, 1.2]:
        zoomed = cv2.resize(image, None, fx=zoom_factor, fy=zoom_factor)
        zh, zw = zoomed.shape
        zh_crop = (zh - h) // 2
        zw_crop = (zw - w) // 2
        cropped_zoom = zoomed[zh_crop:zh_crop+h, zw_crop:zw_crop+w]
        cv2.imwrite(os.path.join(augmented_path, f"{base_name}_zoom{int(zoom_factor*10)}.jpg"), cropped_zoom)

    # Brightness adjustment
    for factor in [0.8, 1.2]:  # darker and brighter
        bright = np.clip(image * factor, 0, 255).astype(np.uint8)
        cv2.imwrite(os.path.join(augmented_path, f"{base_name}_bright{int(factor*10)}.jpg"), bright)

# Process all grayscale images
for count, filename in enumerate(tqdm(os.listdir(input_path))):
    if filename.endswith((".jpg", ".png")):
        img_path = os.path.join(input_path, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        augment_and_save(img, filename, count)

print("Augmentation complete. Check:", augmented_path)

100%|██████████| 1327/1327 [01:46<00:00, 12.52it/s]

Augmentation complete. Check: /content/drive/MyDrive/preprocessing prototype 2/train/Augmented/Autistic


In [ ]:
#cnn model training

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import cv2

# Define dataset paths
AUTISTIC_PATH = "/content/drive/MyDrive/CAPSTONE PROCESSED/TRAIN/AUTISTIC"
HEALTHY_PATH = "/content/drive/MyDrive/CAPSTONE PROCESSED/TRAIN/NON_AUTISTIC"

# Image parameters
IMG_SIZE = 224  # Resize images to 224x224
BATCH_SIZE = 32

# Function to load images and labels
def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        if filename.endswith((".jpg", ".png")):
            img_path = os.path.join(folder, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Convert to grayscale
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))  # Resize to 224x224
            images.append(img)
            labels.append(label)
    return np.array(images), np.array(labels)

# Load data
autistic_images, autistic_labels = load_images_from_folder(AUTISTIC_PATH, 1)  # Label 1 for autistic
healthy_images, healthy_labels = load_images_from_folder(HEALTHY_PATH, 0)  # Label 0 for healthy

# Combine datasets
X = np.concatenate((autistic_images, healthy_images), axis=0)
y = np.concatenate((autistic_labels, healthy_labels), axis=0)

# Normalize images (scale pixel values to 0-1)
X = X / 255.0
X = np.expand_dims(X, axis=-1)  # Add channel dimension (224, 224, 1)

# Shuffle dataset
from sklearn.utils import shuffle
X, y = shuffle(X, y, random_state=42)

# Split dataset into train & validation
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# CNN Model
model = keras.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),  # Dropout to prevent overfitting
    layers.Dense(1, activation='sigmoid')  # Binary classification output
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val), batch_size=BATCH_SIZE)

# Save the trained model
model.save("/content/drive/MyDrive/CAPSTONE_MODEL/AUTISM_MODEL_MORE_EPOCS.h5",include_optimizer=True)

# Evaluate the model
val_loss, val_acc = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {val_acc:.4f}")


In [ ]:
# Best model (with Leaky_Relu activation function)

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import cv2

# Define dataset paths
AUTISTIC_PATH = "/content/drive/MyDrive/CAPSTONE PROCESSED/TRAIN/AUTISTIC"
HEALTHY_PATH = "/content/drive/MyDrive/CAPSTONE PROCESSED/TRAIN/NON_AUTISTIC"

# Image parameters
IMG_SIZE = 224  # Resize images to 224x224
BATCH_SIZE = 32

# Function to load images and labels
def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        if filename.endswith((".jpg", ".png")):
            img_path = os.path.join(folder, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Convert to grayscale
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))  # Resize to 224x224
            images.append(img)
            labels.append(label)
    return np.array(images), np.array(labels)

# Load data
autistic_images, autistic_labels = load_images_from_folder(AUTISTIC_PATH, 1)  # Label 1 for autistic
healthy_images, healthy_labels = load_images_from_folder(HEALTHY_PATH, 0)  # Label 0 for healthy

# Combine datasets
X = np.concatenate((autistic_images, healthy_images), axis=0)
y = np.concatenate((autistic_labels, healthy_labels), axis=0)

# Normalize images (scale pixel values to 0-1)
X = X / 255.0
X = np.expand_dims(X, axis=-1)  # Add channel dimension (224, 224, 1)

# Shuffle dataset
from sklearn.utils import shuffle
X, y = shuffle(X, y, random_state=42)

# Split dataset into train & validation
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# CNN Model with Leaky ReLU activation
model = keras.Sequential([
    layers.Conv2D(32, (3,3), input_shape=(IMG_SIZE, IMG_SIZE, 1)),
    layers.LeakyReLU(alpha=0.01),  # Leaky ReLU instead of ReLU
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, (3,3)),
    layers.LeakyReLU(alpha=0.01),  # Leaky ReLU instead of ReLU
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(128, (3,3)),
    layers.LeakyReLU(alpha=0.01),  # Leaky ReLU instead of ReLU
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dense(128),
    layers.LeakyReLU(alpha=0.01),  # Leaky ReLU instead of ReLU
    layers.Dropout(0.5),  # Dropout to prevent overfitting
    layers.Dense(1, activation='sigmoid')  # Binary classification output
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val), batch_size=BATCH_SIZE)

# Save the trained model
model.save("/content/drive/MyDrive/CAPSTONE_MODEL/AUTISM_MODEL_LEAKY_RELU_MORE_EPOCS.h5", include_optimizer=True)

# Evaluate the model
val_loss, val_acc = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {val_acc:.4f}")


In [ ]:
# CNN with preprocessed data

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import cv2

# Define dataset paths
AUTISTIC_PATH = "/content/drive/MyDrive/preprocessing prototype/Autistic"
HEALTHY_PATH = "/content/drive/MyDrive/preprocessing prototype/Non_autistic"

# Image parameters
IMG_SIZE = 224
BATCH_SIZE = 32

# Function to load images and labels
def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        if filename.endswith(('.jpg', '.png')):
            img_path = os.path.join(folder, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            images.append(img)
            labels.append(label)
    return np.array(images), np.array(labels)

# Load data
autistic_images, autistic_labels = load_images_from_folder(AUTISTIC_PATH, 1)
healthy_images, healthy_labels = load_images_from_folder(HEALTHY_PATH, 0)

# Combine datasets
X = np.concatenate((autistic_images, healthy_images), axis=0)
y = np.concatenate((autistic_labels, healthy_labels), axis=0)

# Ensure images are in correct shape
X = np.expand_dims(X, axis=-1)

# Shuffle dataset
X, y = shuffle(X, y, random_state=42)

# Split dataset into train & validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# CNN Model
model = keras.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), batch_size=BATCH_SIZE)

# Save the trained model
model.save("/content/drive/MyDrive/CAPSTONE_MODEL/AUTISM_MODEL_PROCESSED.h5", include_optimizer=True)

# Evaluate the model
val_loss, val_acc = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {val_acc:.4f}")

In [ ]:
# Testing

In [ ]:
import os
import numpy as np
import cv2
from tensorflow import keras

# Load the model
model = keras.models.load_model('/content/drive/MyDrive/CAPSTONE_MODEL/AUTISM_MODEL_SWISH.h5')

# Folder path containing images
input_folder = '/content/drive/MyDrive/CAPSTONE_DATASETS/Autism_Images/data/test/Non_Autistic'

# Image processing parameters
IMG_SIZE = 224

# Counters for predictions
down_syndrome_count = 0
healthy_count = 0

# Process each .jpg image in the folder
for filename in os.listdir(input_folder):
    if filename.lower().endswith('.jpg'):  # Only read .jpg files
        img_path = os.path.join(input_folder, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

        if img is None:
            print(f"Error loading image: {filename}")
            continue

        # Preprocess image
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        img = img / 255.0
        img = np.expand_dims(img, axis=(0, -1))  # Add batch and channel dimensions

        # Predict
        prediction = model.predict(img)
        label = 'Down Syndrome' if prediction[0][0] > 0.5 else 'Healthy'
        print(f"Image: {filename}, Prediction: {label}")

        # Update count
        if label == 'Down Syndrome':
            down_syndrome_count += 1
        else:
            healthy_count += 1

# Final result
print(f"\nTotal Down Syndrome Predictions: {down_syndrome_count}")
print(f"Total Healthy Predictions: {healthy_count}")
temp = healthy_count/(healthy_count + down_syndrome_count);
print(f"Accuracy : {temp*100}%")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 512ms/step
Image: 134.jpg, Prediction: Down Syndrome
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
Image: 068.jpg, Prediction: Healthy
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Image: 099.jpg, Prediction: Healthy
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Image: 116.jpg, Prediction: Healthy
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Image: 136.jpg, Prediction: Healthy
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Image: 109.jpg, Prediction: Healthy
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Image: 128.jpg, Prediction: Healthy
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Image: 123.jpg, Prediction: Healthy
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Image: 066.jpg, Prediction: Healthy
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Image: 065.jpg, Prediction: Healthy
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Image: 069.jpg, Prediction: Healthy
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Image: 063.jpg, Prediction: Healthy
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Image: 074.jpg, Prediction: Down Syndrome
1/1 ━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
# Count number of photos

In [ ]:
# List files